<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_groupchat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="콜랩에서 열기"/></a>

# 자동 생성된 상담원 채팅: 그룹 채팅

자동 생성은 자동화된 채팅을 통해 공동으로 작업을 수행하는 데 사용할 수 있는 LLM, 툴 또는 사람으로 구동되는 대화 가능 에이전트를 제공합니다. 이 프레임워크는 여러 상담원이 대화를 통해 툴을 사용하고 사람이 참여할 수 있도록 지원합니다.
이 기능에 대한 설명서는 [여기](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat)를 참조하세요.

이 노트북은 https://github.com/microsoft/FLAML/blob/4ea686af5c3e8ff24d9076a7a626c8b28ab5b1d7/notebook/autogen_multiagent_roleplay_chat.ipynb 을 기반으로 수정되었습니다.

## 요구 사항

자동 생성에는 `Python>=3.8`이 필요합니다. 이 노트북 예제를 실행하려면 설치하세요:
```bash
pip 설치 pyautogen
```

In [105]:
%%capture --no-stderr
# %pip install pyautogen~=0.1.0

## API 엔드포인트 설정

`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) 함수는 환경 변수 또는 json 파일에서 구성 목록을 로드합니다.

In [106]:
import autogen

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)
# config_list_gpt35 = autogen.config_list_from_json(
#     "OAI_CONFIG_LIST",
#     filter_dict={
#         "model": {
#             "gpt-3.5-turbo",
#             "gpt-3.5-turbo-16k",
#             "gpt-3.5-turbo-0301",
#             "chatgpt-35-turbo-0301",
#             "gpt-35-turbo-v0301",
#         },
#     },
# )

먼저 유효한 json 문자열이어야 하는 환경 변수 "OAI_CONFIG_LIST"를 찾습니다. 해당 변수를 찾을 수 없으면 "OAI_CONFIG_LIST"라는 이름의 json 파일을 찾습니다. 이 파일은 모델별로 구성을 필터링합니다(다른 키로도 필터링할 수 있음). 필터 조건에 따라 gpt-4 모델만 목록에 유지됩니다.

구성 목록은 다음과 같습니다:
```python
config_list = [
    {
        '모델': 'gpt-4',
        'api_key': '<귀하의 OpenAI API 키는 여기>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<귀하의 Azure OpenAI API 키는 여기>',
        'api_base': '<귀하의 Azure OpenAI API 베이스는 여기>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k',
        'api_key': '<귀하의 Azure OpenAI API 키는 여기>',
        'api_base': '<귀하의 Azure OpenAI API 베이스는 여기>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

콜랩에서 이 노트북을 열면 왼쪽 패널의 파일 아이콘을 클릭한 다음 '파일 업로드' 아이콘을 선택해 파일을 업로드할 수 있습니다.

YAML 파일에서 불러오기 등 원하는 다른 방법으로 config_list의 값을 설정할 수 있습니다.

## 에이전트 생성

In [107]:
llm_config = {"config_list": config_list_gpt4, "seed": 42}
user_proxy = autogen.UserProxyAgent(
   name="User_proxy",
   system_message="A human admin.",
   code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
   human_input_mode="TERMINATE"
)
coder = autogen.AssistantAgent(
    name="Coder",
    llm_config=llm_config,
)
pm = autogen.AssistantAgent(
    name="Product_manager",
    system_message="Creative in software product ideas.",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(agents=[user_proxy, coder, pm], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

## 채팅 시작

In [108]:
user_proxy.initiate_chat(manager, message="Find a latest paper about gpt-4 on arxiv and find its potential applications in software.")
# type exit to terminate the chat

User_proxy (to chat_manager):

Find a latest paper about gpt-4 on arxiv and find its potential applications in software.

--------------------------------------------------------------------------------
Coder (to chat_manager):

To find the latest paper about GPT-4 on arxiv, I'll provide you with a Python code that fetches the most recent papers from the arxiv API and filters the results to get the most relevant paper related to GPT-4. After fetching the paper, I'll extract the information for potential applications in software. Please execute the following Python code:

```python
import requests
from bs4 import BeautifulSoup
import re

def fetch_arxiv_papers(query):
    base_url = "http://export.arxiv.org/api/query?"
    search_query = "all:" + query
    response = requests.get(base_url, params={"search_query": search_query, "sortBy": "submittedDate", "sortOrder": "descending"})
    return BeautifulSoup(response.content, "xml")

def find_gpt4_paper():
    papers = fetch_arxiv_papers("